In [1]:
# import dependencies
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
#from MulticoreTSNE import MulticoreTSNE as TSNE

In [2]:
import os
os.chdir('/data/work/final/06_scenic/T_cell')

In [3]:
import os, sys
os.getcwd()
os.listdir(os.getcwd())
import loompy as lp;
import numpy as np;
import scanpy as sc;
x=sc.read_csv("sce_exp_hvg5000.csv");#R中导出的表达矩阵
row_attrs = {"Gene": np.array(x.var_names),};
col_attrs = {"CellID": np.array(x.obs_names)};
lp.create("sce.loom",x.X.transpose(),row_attrs,col_attrs)

In [9]:
# transcription factors list
f_tfs = "/data/work/final/06_scenic/T_cell/allTFs_hg38.txt" # human
# f_tfs = "/ddn1/vol1/staging/leuven/stg_00002/lcb/cflerin/resources/allTFs_dmel.txt" # drosophila
# f_tfs = "/ddn1/vol1/staging/leuven/stg_00002/lcb/cflerin/resources/allTFs_mm.txt"   # mouse
# tf_names = load_tf_names( f_tfs )


In [10]:
f_loom_path_scenic='/data/work/final/06_scenic/T_cell/sce.loom'

In [11]:
##step1 GRN---运行完得到sce.adj.csv文件
!pyscenic grn --num_workers 60 \
  --sparse \
  --method grnboost2 \
  --output sce.adj.csv \
  sce.loom \
  allTFs_hg38.txt
adjacencies = pd.read_csv("sce.adj.csv", index_col=False, sep=',')
adjacencies.head()


2025-06-10 23:32:27,159 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2025-06-10 23:32:33,971 - pyscenic.cli.pyscenic - INFO - Inferring regulatory networks.
preparing dask client
parsing input
creating dask graph
60 partitions
computing dask graph
not shutting down client, client was created externally
finished
2025-06-10 23:49:30,538 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nbytes = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/distributed/worker.py", line 1237, in heartbeat
    response = await retry_operation(
  File "/opt/conda/lib/python3.8/site-packages/distributed/utils_comm.py", line 434, in 

,TF,target,importance
0,ZNF627,MMP9,1759.169584
1,ZNF627,ACP5,1352.229479
2,ZNF627,CTSK,1176.569884
3,ZNF627,AC131944.1,1085.737354
4,ZNF627,APOE,1052.343886


In [ ]:
#step2 RcisTarget---运行完得到sce.regulons.csv文件
import glob
# ranking databases
f_db_glob = "/data/work/paper/scenic/database/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather"
#f_db_names = ' '.join( glob.glob(f_db_glob) )

# motif databases
f_motif_path = "/data/work/paper/scenic/database/motifs-v9-nr.hgnc-m0.001-o0.0.tbl"

!pyscenic ctx --num_workers 60 \
  --output sce.regulons.csv \
  --expression_mtx_fname sce.loom \
  --all_modules \
  --mask_dropouts \
  --mode "dask_multiprocessing" \
  --min_genes 10 \
  --annotations_fname motifs-v9-nr.hgnc-m0.001-o0.0.tbl \
  sce.adj.csv \
  hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
  #这一步的目的
  #进行TF-motif富集分析，识别直接靶标
  #得到转录因子(TF)与其对应的直接作用的靶点,称为regulon(每一个regulon是1个TF和其调控的靶基因)


2025-06-10 23:49:39,935 - pyscenic.cli.pyscenic - INFO - Creating modules.

2025-06-10 23:49:40,006 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2025-06-10 23:49:48,033 - pyscenic.utils - INFO - Calculating Pearson correlations.

2025-06-10 23:49:48,056 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2025-06-10 23:49:53,691 - pyscenic.utils - INFO - Creating modules.

2025-06-10 23:50:24,541 - pyscenic.cli.pyscenic - IN

In [14]:
#step3 AUCell---运行完得到sce_SCENIC.loom文件，即分析结果
!pyscenic aucell --num_workers 3 \
  --output sce_SCENIC.loom \
  sce.loom \
  sce.regulons.csv
  #这一步的目的
  #使用AUCell对每个细胞的每个regulon活性进行评分。


2025-06-11 17:09:52,949 - pyscenic.cli.pyscenic - INFO - Loading expression matrix.

2025-06-11 17:10:01,102 - pyscenic.cli.pyscenic - INFO - Loading gene signatures.
Create regulons from a dataframe of enriched features.
Additional columns saved: []

2025-06-11 17:10:01,210 - pyscenic.cli.pyscenic - INFO - Calculating cellular enrichment.

2025-06-11 17:11:13,831 - pyscenic.cli.pyscenic - INFO - Writing results to file.
/opt/conda/lib/python3.8/site-packages/pyscenic/cli/utils.py:347: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, threshold in auc_thresholds.iteritems()


In [15]:
print('finish')

finish


In [ ]:
#database的下载地址
wget https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg38/refseq_r80/mc_v10_clust/gene_based/hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather
wget https://resources.aertslab.org/cistarget/motif2tf/motifs-v9-nr.hgnc-m0.001-o0.0.tbl